In [ ]:
function res_colocacao(A,L,grid_k,grid_z,P,r,μ)
    nz = length(grid_z)
    nl = length(L)
    nk = length(grid_k)
    res = zeros(nk,nz)
    k_prov = zeros(nk)

    pe =  phi_matricial(L,grid_k) # termos da colocação
    c_0 = fea_consumo(A,pe) # consumo
    k_1 = grid_z' .+ (1 + r).*grid_k .- c_0

    #k_prov = sum(k_1,dims = 2)/nz
    #phi1 = phi_matricial(L,k_prov)

    for estado in 1:nz
        phi1 = phi_matricial(L,k_1[:,estado])
        c_1 = fea_consumo(A[:,estado],phi1)
        ulinha = c_1.^(-μ)
        dcdk = ones(nz)'*(1+r)
        lde = ulinha.*dcdk
        for id in 1:nl
            res[id,estado] = c_0[id,estado]^(-μ) - 0.96*dot(P[estado,:],lde[id,:])
        end
    end
    
    return res
end



In [ ]:
function fea_colocacao(L,grid_k,grid_z,P,r,μ)
    A = Vector(LinRange(0.5,1.2,length(L)))
    for i in 1:(length(grid_z)-1)
       A= hcat(A,Vector(LinRange(0.5 + i/20,1.2+ i/90,length(L))))
    end
    g(A) = res_colocacao(A,L,L,grid_z,P,r,μ)
    a_otimo = nlsolve(g,A).zero

    c_0 = fea_consumo(a_otimo, phi_matricial(L,grid_k))
    k_1 = grid_z' .+ (1 +r).*grid_k .- c_0
    
    return c_0, k_1
end

In [ ]:
function eee_colocacao(a_otimo,L,grid_k,grid_z,P,r)
    nk = length(grid_k)
    nz = length(grid_z)
    eee = zeros(nk,nz)

    pe = phi_matricial(L,grid_k)
    c_0 = fea_consumo(a_otimo,pe)

    k_1 = grid_z' .+ (1 + r).*grid_k .- c_0
    for estado in 1:nz
        phi1 = phi_matricial(L,k_1[:,estado])
        c_1 = fea_consumo(a_otimo[:,estado],phi1)
        ulinha = c_1.^(-1.0001)
        dcdk = 1+r 
        lde = ulinha.*dcdk
        for id in 1:nk
             inversa = (0.96*dot(P[estado,:],lde[id,:]))^(-1/1.0001)
             eee[id,estado] = 1 - inversa/c_0[id,estado]
        end
    end
    eee = log10.(abs.(eee))
    return eee
end